In [1]:
import dask.dataframe as ddf
import pandas as pd
import numpy as np

/home/ubuntu/.local/lib/python3.8/site-packages/dask/dataframe/utils.py:374: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  _numeric_index_types = (pd.Int64Index, pd.Float64Index, pd.UInt64Index)
/home/ubuntu/.local/lib/python3.8/site-packages/dask/dataframe/utils.py:374: FutureWarning: pandas.Float64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  _numeric_index_types = (pd.Int64Index, pd.Float64Index, pd.UInt64Index)
/home/ubuntu/.local/lib/python3.8/site-packages/dask/dataframe/utils.py:374: FutureWarning: pandas.UInt64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  _numeric_index_types = (pd.Int64Index, pd.Float64Index, pd.UInt64Index)


In [2]:
graph = pd.read_parquet('result/global_RG')
all_users = list(set(graph.index.to_list() + graph.columns.to_list()))
res = {}

In [4]:
# create the list for comparsion, add up columns and rows for each user
def revert_list(user):
    col_len = len(graph)
    cols = []
    row_len = len(graph.columns)
    rows = []
    if user not in graph.index:
        rows = [0.0] * row_len
    else:
        rows = graph.loc[user].to_list()
    if user not in graph:
        cols = [0.0] * col_len
    else:
        cols = graph[user].to_list()
    res[user] = rows + cols

In [5]:
result = [revert_list(i) for i in all_users]
from sklearn.metrics.pairwise import cosine_similarity
num_users = len(all_users)

In [6]:
sim_matrix = np.empty((num_users,num_users))
# np.fill_diagonal(sim_matrix,np.nan)
np.fill_diagonal(sim_matrix,1)

In [7]:
for i in range(num_users - 1):
    for j in range(i + 1, num_users):
        u1 = all_users[i]
        u2 = all_users[j]
        sim = cosine_similarity([res[u1]],[res[u2]])
        sim_matrix[i][j] = sim
        sim_matrix[j][i] = sim

In [8]:
df = pd.DataFrame(sim_matrix,index=all_users,columns=all_users)

In [9]:
df.to_parquet('result/user-similarity')